# Explingo Experiment Runner

This notebook:
1. Loads the gold-standard dataset, prepares the metrics functions, and verifies that the metric functions give the maximum score on the gold-standard dataset, and lower scores on less aligned datasets
2. Runs the prompt-design, few-shot, and bootstrap-few-shot experiments on a testing dataset

## Setup
Import necessary libraries and prepare the LLM

**Note: To run these cells, you need a `keys.yaml` file in the top-level Explingo directory with the following line:**
```yaml
openai_api_key: <your_openai_api_key>
```

In [8]:
import pandas as pd

from experiment_runner import ExplingoExperimentRunner
import os
import yaml
import dspy
import metrics
import random

In [9]:
with open(os.path.join("..", "keys.yaml"), "r") as file:
    config = yaml.safe_load(file)
    openai_api_key = config["openai_api_key"]

llm = dspy.OpenAI(model='gpt-4o', api_key=openai_api_key, max_tokens=1000)

Now, we create the main experiment runner object. This object takes in a dataset, and then
1. Splits the dataset into a training dataset and a testing dataset (see notes below)
2. Sets up the evaluation metrics (see notes below). The fluency metric is set up to use sample from the dataset as reference
3. Runs the experiments on the testing dataset

Some examples in the testing datasets include gold-standard narratives; others include only a sample explanation.
- The former makes up the gold-standard dataset used for tuning the evaluation metrics and providing few-shot examples.
- The latter makes up the testing dataset used for evaluation and for bootstrapping few-shot examples

We use the following metrics, all scored on a scale from 0-4:
- Accuracy: the narrative accurately describes the information in the explanation
- Fluency: the narrative is coherent and natural, as compared to the gold-standard explanations. We pass in a small list of sample narratives from the gold-standard dataset to compare against
- Conciseness: the narrative is not too long, as compared to the gold-standard explanations. For now, any narrative that is no longer than the longest gold-standard narrative will score 4
- Completeness: the narrative includes all relevant information from the original explanation 

**Note: You can set `verbose=1` to see the narratives generated, or `verbose=2` to see the explanations, narratives, and rationalizations**

In [10]:
# iterate all datasets in the eval_data folder
runners = {}
total_eval = 0
for dataset in os.listdir(os.path.join("eval_data")):
    runners[dataset] = ExplingoExperimentRunner(llm=llm, openai_api_key=openai_api_key, dataset_filepath = os.path.join("eval_data", dataset), verbose=1)
    total_eval += len(runners[dataset].eval_data)
    
print("Total eval examples:", total_eval)
results = []

eval_data\housing_1.json
Total number of examples: 35
Labeled training examples: 5
Labeled evaluation examples: 15
Unlabeled training examples: 5
Unlabeled evaluation examples: 10
Max optimal length: 12.24
---
eval_data\housing_2.json
Total number of examples: 22
Labeled training examples: 5
Labeled evaluation examples: 7
Unlabeled training examples: 5
Unlabeled evaluation examples: 5
Max optimal length: 9.0
---
eval_data\housing_3.json
Total number of examples: 22
Labeled training examples: 5
Labeled evaluation examples: 8
Unlabeled training examples: 5
Unlabeled evaluation examples: 4
Max optimal length: 6.84
---
eval_data\mushroom_1.json
Total number of examples: 30
Labeled training examples: 5
Labeled evaluation examples: 6
Unlabeled training examples: 5
Unlabeled evaluation examples: 14
Max optimal length: 9.9
---
eval_data\mushroom_2.json
Total number of examples: 30
Labeled training examples: 5
Labeled evaluation examples: 6
Unlabeled training examples: 5
Unlabeled evaluation ex

## Basic prompt design experiment

We begin with basic prompts. With 4 metrics (without completeness), each with a score of 0-2, the maximum score is 8. 

We generate narratives/rationalizations on `max_iters=5` sample explanations, and return the average total score.

In [15]:
# Utilities for cleaner results

def pretty_print(result):
    s = f"Total score: {result[0]}"
    s2 = ", ".join([f"{k}: {v}" for k, v in result[1].items()])
    print(f"{s} ({s2})")
    
def update_results(method, dataset, scores, kwargs):
    result = {"dataset": dataset, "total score": scores[0]}
    result.update(scores[1])
    result.update(kwargs)
    results.append(result)

In [12]:
prompts = ["You are helping users understand an ML model's prediction. Given an explanation and information about the model, "
           "convert the explanation into a human-readable narrative.",
           "You are helping users who do not have experience working with ML understand an ML model's prediction. Given an explanation and information about the model, convert the explanation into a human-readable narrative. Make your answers sound as natural as possible.",
           "You are helping users understand an ML model's prediction. Given an explanation and information about the model, convert the explanation into a human-readable narrative. Be sure to explicitly mention all values from the explanation in your response.",
]

# for dataset in runners:
#     runner = runners[dataset]
#     print(f"Dataset: {dataset}")
#     for prompt in prompts:
#         print(f"Prompt: {prompt}")
#         scores = runner.run_basic_prompting_experiment(prompt=prompt, max_iters=5)
#         update_results("basic_prompting", dataset, scores, {"prompt": prompt})
#         pretty_print(scores)
#         print("--")
#     print("=====")

## Few-shot experiment

Next, we repeat the experiment with the addition of N few-shot examples from the gold-standard dataset.

In [6]:
for dataset in runners:
    runner = runners[dataset]
    print(f"Dataset: {dataset}")
    for i in [1, 3, 5]:
        print(f"Few-shot n: {i}")
        scores = runner.run_few_shot_experiment(n_few_shot=i, prompt=prompts[0], max_iters=5)
        update_results("few_shot", dataset, scores, {"n_few_shot": i, "prompt": prompts[0]})
        pretty_print(scores)
        print("--")
    print("=====")

Dataset: housing_1.json
Few-shot n: 1
Narrative: The house's location in NoRidge increased the predicted price by about $23,000. The relatively larger above ground living space increased the price by about $20,000. The larger than average second floor increased the price by about $18,000. The higher-than-average material and finish rating increased the price by about $9,600. The house is newer than average, with a construction year of 2000, which increased the price by about $8,200.
Total Score: 3.5555555555555554
conciseness: 3.5555555555555554, 
--
Narrative: The house's wood foundation reduced the predicted price by about $18,650. The house's location in Mitchel reduced the price by about $13,500. The lower-than-average material rating also reduced the house price by about $10,700. The relatively larger three season porch area increased the price by about $10,000. Having only one bedroom above ground increased the price by about $8,900.
Total Score: 4
conciseness: 4, 
--
Narrative: 

## Bootstrapped few-shot
Next, we repeat the experiment with the addition of 3 examples bootstrapped by DSPy to optimize the evaluation metrics.

In [16]:
for dataset in runners:
    runner = runners[dataset]
    print(f"Dataset: {dataset}")
    for i, j in [[0, 1], [0, 3], [3, 3]]:
        print(f"Few-shot n: {i}, Bootstrapped n: {j}")
        scores = runner.run_bootstrap_few_shot_experiment(n_labeled_few_shot=i, n_bootstrapped_few_shot=j, max_iters=5)
        update_results("bootstrap_few_shot", dataset, scores, {"n_few_shot": i, "n_bootstrapped_few_shot": j, "prompt": prompts[0]})
        pretty_print(scores)
        print("--")

Dataset: housing_1.json
Few-shot n: 0, Bootstrapped n: 1


  0%|          | 0/10 [00:00<?, ?it/s]

Narrative: The model predicts the house price based on several factors. The above ground living area, which is 1256 square feet, decreases the predicted price by $12,527.46. The overall material and finish of the house, rated at 5, further reduces the price by $10,743.76. The absence of a second floor contributes to a decrease of $10,142.29. The house's location within the Edwards neighborhood in Ames city limits lowers the price by $9,913.81. However, the presence of a wood deck area measuring 736 square feet increases the predicted price by $9,846.38.
Total Score: 14.183006535947712
accuracy: 4.0, completeness: 4.0, fluency: 4.0, conciseness: 2.1830065359477127, 
--
Narrative: The model predicts the house price based on several key features. The second floor square feet, which is 854.00, contributes positively with a value of 12757.84. The original construction date of the house, which is 2003.00, also adds positively with a contribution of 9115.72. However, the total square feet of 

 50%|█████     | 5/10 [00:00<00:00, 47.81it/s]

Narrative: The model predicts the house price based on several factors. The above grade (ground) living area of 1256 square feet decreases the predicted price by $12,527.46. The overall material and finish rating of the house, which is 5, reduces the price by $10,743.76. The absence of a second floor (0 square feet) lowers the price by $10,142.29. The house's location within the Edwards neighborhood in Ames city limits decreases the price by $9,913.81. However, the presence of a wood deck area of 736 square feet increases the predicted price by $9,846.38.
Total Score: 15.043572984749456
accuracy: 4.0, completeness: 4.0, fluency: 4.0, conciseness: 3.0435729847494555, 
--
Narrative: The model predicts the house price based on several key features. The second floor square feet, which is 854.00, contributes positively with a value of 12757.84. The original construction date of the house, which is 2003.00, also adds positively with a contribution of 9115.72. However, the total square feet o

...

Narrative: The model predicts that the student is less likely to pass their class because they do not have family educational support, which negatively impacts the prediction by 2.26 points. Additionally, the student is not in a romantic relationship, which positively impacts the prediction by 1.11 points. Lastly, the student's gender is male, which slightly negatively impacts the prediction by 0.60 points.
Total Score: 12.0
accuracy: 4.0, completeness: 4.0, fluency: 4.0, conciseness: 0.0, 
--
Narrative: The presence of family educational support and the lack of a romantic relationship suggest the student is more likely to pass the class. However, being 17 years old slightly decreases the probability of passing.
Total Score: 15.277777777777779
accuracy: 4.0, completeness: 4.0, fluency: 4.0, conciseness: 3.2777777777777777, 
--
Narrative: Being in a romantic relationship, the lack of family support, and the sex (male) suggest the student is less likely to pass the class.
Total Score: 16

 50%|█████     | 5/10 [00:00<00:00, 44.05it/s]

Narrative: The lack of a romantic relationship and the presence of family educational support suggest the student is more likely to pass the class. However, the lower quality of family relationships indicates a lower probability of passing.
Total Score: 16.0
accuracy: 4.0, completeness: 4.0, fluency: 4.0, conciseness: 4, 
--
Narrative: The lack of a romantic relationship, the presence of family educational support, and a high quality of family relationships suggest the student is more likely to pass the class.
Total Score: 16.0
accuracy: 4.0, completeness: 4.0, fluency: 4.0, conciseness: 4, 
--


 70%|███████   | 7/10 [00:00<00:00, 41.77it/s]


Bootstrapped 3 full traces after 8 examples in round 0.
Narrative: The lack of family educational support and not having attended nursery school suggest the student is less likely to pass the class. However, the lack of a romantic relationship indicates a higher probability of passing.
Total Score: 15.13888888888889
accuracy: 4.0, completeness: 4.0, fluency: 4.0, conciseness: 3.1388888888888884, 
--
Total score: 14.511111111111111 (accuracy: 4.0, completeness: 3.2, fluency: 4.0, conciseness: 3.311111111111111)
--


In [17]:
 result_df = pd.DataFrame(results)
result_df.to_csv("results.csv")
result_df

,dataset,total score,accuracy,completeness,fluency,conciseness,n_few_shot,n_bootstrapped_few_shot,prompt
0,housing_1.json,13.087582,4.0,4.0,2.8,2.287582,0,1,You are helping users understand an ML model's...
1,housing_1.json,13.087582,4.0,4.0,2.8,2.287582,0,3,You are helping users understand an ML model's...
2,housing_1.json,15.605229,4.0,4.0,4.0,3.605229,3,3,You are helping users understand an ML model's...
3,housing_2.json,10.444444,4.0,4.0,2.0,0.444444,0,1,You are helping users understand an ML model's...
4,housing_2.json,10.444444,4.0,4.0,2.0,0.444444,0,3,You are helping users understand an ML model's...
5,housing_2.json,14.875556,4.0,4.0,3.8,3.075556,3,3,You are helping users understand an ML model's...
6,housing_3.json,9.600000,4.0,4.0,1.6,0.000000,0,1,You are helping users understand an ML model's...
7,housing_3.json,9.600000,4.0,4.0,1.6,0.000000,0,3,You are helping users understand an ML model's...
8,housing_3.json,15.002339,4.0,3.6,3.8,3.602339,3,3,You are helping users understand an ML model's...
9,mushroom_1.json,10.358923,4.0,4.0,1.4,0.958923,0,1,You are helping users understand an ML model's...
